# Handling Date - Time Features in Dataset 
- Handling date & time data in Pandas is very important for Machine Learning because many datasets contain timestamps (sales data, flights, stock prices, sensor data, etc.). Proper preprocessing of datetime features helps models learn patterns like seasonality, trends, and time-based behavior.

| Task | Pandas Method |
|------|---------------|
| Parse date string | pd.to_datetime(df[col]) |
| Extract year | .dt.year |
| Extract month | .dt.month |
| Extract day | .dt.day |
| Day of week (0=Mon) | .dt.dayofweek |
| Day name string | .dt.day_name() |
| ISO week of year | .dt.isocalendar().week |
| Quarter (1-4) | .dt.quarter |
| Hour / Minute / Second | .dt.hour, .dt.minute, .dt.second |
| Time component (HH:MM:SS) | .dt.time |
| Days elapsed | (now - df[col]).dt.days |
| Elapsed in any unit | (now - df[col]) / np.timedelta64(1, s/m/h) |

In [22]:
import pandas as pd
import numpy as np
import datetime

In [23]:
date = pd.read_csv("orders.csv")
time = pd.read_csv("messages.csv")

In [24]:
date.head()

,date,product_id,city_id,orders
0,2019-12-10,5628,25,3
1,2018-08-15,3646,14,157
2,2018-10-23,1859,25,1
3,2019-08-17,7292,25,1
4,2019-01-06,4344,25,3


In [25]:
time.head()

,date,msg
0,2013-12-15 00:50:00,ищу на сегодня мужика 37
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше


In [26]:
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   date        1000 non-null   object
 1   product_id  1000 non-null   int64 
 2   city_id     1000 non-null   int64 
 3   orders      1000 non-null   int64 
dtypes: int64(3), object(1)
memory usage: 31.4+ KB


In [27]:
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1000 non-null   object
 1   msg     1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


#### Converting Date [ Object ] Column into Date_Time 

In [29]:
# Convert the 'date' column in orders from object -> datetime64[ns]
# pd.to_datetime() automatically infers common date formats (YYYY-MM-DD, etc.)
# After conversion the .dt accessor becomes available for feature extraction
date['date'] = pd.to_datetime(date['date'])
date.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   date        1000 non-null   datetime64[ns]
 1   product_id  1000 non-null   int64         
 2   city_id     1000 non-null   int64         
 3   orders      1000 non-null   int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 31.4 KB


---
##  Date Feature Extraction

Once a column is datetime64, we unlock the .dt accessor to extract individual date components. Each extracted component becomes a new column usable as an independent ML feature.

### Extracting Year

In [31]:
# Extract the year as an integer (e.g., 2018, 2019)
# Useful for capturing long-term trends across years
date['date_year'] = date['date'].dt.year
date.head(2)

,date,product_id,city_id,orders,date_year
0,2019-12-10,5628,25,3,2019
1,2018-08-15,3646,14,157,2018


#### Extracting Month

In [33]:
# Extract the month as an integer (1=January, 12=December)
# Key feature for seasonal analysis - sales, demand, weather all show monthly patterns
date['date_month'] = date['date'].dt.month
date.head(2)

,date,product_id,city_id,orders,date_year,date_month
0,2019-12-10,5628,25,3,2019,12
1,2018-08-15,3646,14,157,2018,8


#### Extracting Day

In [35]:
# Extract the day of the month (1-31)
# Useful for spotting billing cycle patterns or month-end order spikes
date['date_day'] = date['date'].dt.day
date.head(2)

,date,product_id,city_id,orders,date_year,date_month,date_day
0,2019-12-10,5628,25,3,2019,12,10
1,2018-08-15,3646,14,157,2018,8,15


#### Extracting Day of Week (Numeric)

> Note: .dt.dayofweek uses 0-based indexing where 0=Monday and 6=Sunday. We add +1 to get a 1-7 scale.

In [37]:
# .dt.dayofweek: 0=Monday, 1=Tuesday, ..., 6=Sunday
# Adding +1 to make it 1-indexed (1=Monday, 7=Sunday)
date['week_of_day'] = date['date'].dt.dayofweek+1
date.head(2)

,date,product_id,city_id,orders,date_year,date_month,date_day,week_of_day
0,2019-12-10,5628,25,3,2019,12,10,2
1,2018-08-15,3646,14,157,2018,8,15,3


#### Extracting Day Name

In [39]:
# .dt.day_name() returns the full English day name ('Monday', 'Tuesday', etc.)
# More human-readable than numeric; great for groupby aggregations and plots
date['day_name'] = date['date'].dt.day_name()
date.head(2)

,date,product_id,city_id,orders,date_year,date_month,date_day,week_of_day,day_name
0,2019-12-10,5628,25,3,2019,12,10,2,Tuesday
1,2018-08-15,3646,14,157,2018,8,15,3,Wednesday


#### Is the Day a Weekend?

Binary (0/1) flags are powerful features for models dealing with consumer behavior, traffic, or sales - many patterns differ significantly between weekdays and weekends.

> We first drop unused columns to keep the DataFrame clean.

In [41]:
# Drop original business columns to simplify the DataFrame for date feature demonstration
# In a real pipeline you would keep these alongside your date features
date = date.drop(columns = ['product_id','city_id','orders'])

In [42]:
# Create a binary weekend flag: 1 if Saturday or Sunday, else 0
# np.where(condition, value_if_true, value_if_false) is a vectorized if-else
date['day_is_weekend'] = np.where(date['day_name'].isin(['Saturday','Sunday']),1,0)
date.head()

,date,date_year,date_month,date_day,week_of_day,day_name,day_is_weekend
0,2019-12-10,2019,12,10,2,Tuesday,0
1,2018-08-15,2018,8,15,3,Wednesday,0
2,2018-10-23,2018,10,23,2,Tuesday,0
3,2019-08-17,2019,8,17,6,Saturday,1
4,2019-01-06,2019,1,6,7,Sunday,1


#### Extracting ISO Week of Year

> We use .dt.isocalendar().week (ISO 8601 standard) instead of the deprecated .dt.weekofyear. ISO weeks start on Monday; Week 1 is the week containing the first Thursday of the year.

In [44]:
# Extract ISO week number of the year (1-53)
# .isocalendar() returns a DataFrame with 'year', 'week', 'day' columns - we take 'week'
# Useful for weekly aggregations and week-based seasonality analysis
date['day_week'] = date['date'].dt.isocalendar().week
date.head(2)

,date,date_year,date_month,date_day,week_of_day,day_name,day_is_weekend,day_week
0,2019-12-10,2019,12,10,2,Tuesday,0,50
1,2018-08-15,2018,8,15,3,Wednesday,0,33


#### Extracting Quarter

In [46]:
# Extract the fiscal/calendar quarter (1-4)
# Q1: Jan-Mar | Q2: Apr-Jun | Q3: Jul-Sep | Q4: Oct-Dec
# Essential feature for financial and business time-series analysis
date['quarter'] =  date['date'].dt.quarter
date.head(2)

,date,date_year,date_month,date_day,week_of_day,day_name,day_is_weekend,day_week,quarter
0,2019-12-10,2019,12,10,2,Tuesday,0,50,4
1,2018-08-15,2018,8,15,3,Wednesday,0,33,3


#### Extracting Semester

A semester divides the year into two halves (no built-in pandas method - we derive from quarter):
- **Semester 1**: Q1 + Q2 (January through June)
- **Semester 2**: Q3 + Q4 (July through December)

In [48]:
# Derive semester from the quarter column using np.where (vectorized conditional)
# Quarters 1 & 2 -> Semester 1 (first half); Quarters 3 & 4 -> Semester 2 (second half)
date['semester'] = np.where(date['quarter'].isin([1,2]),1,2)
date.head(2)

,date,date_year,date_month,date_day,week_of_day,day_name,day_is_weekend,day_week,quarter,semester
0,2019-12-10,2019,12,10,2,Tuesday,0,50,4,2
1,2018-08-15,2018,8,15,3,Wednesday,0,33,3,2


---
## Time Elapsed / Duration Calculation

Computing how long ago something happened is a very common feature engineering task. Subtracting two datetime64 values produces a timedelta64 which can then be converted to any time unit using np.timedelta64.

### Get Todays Timestamp

In [50]:
# Capture the current date and time as the reference point for elapsed calculations
# datetime.datetime.today() returns a naive datetime (no timezone info)
# For timezone-aware data use datetime.datetime.now(tz=...) instead
today = datetime.datetime.today()
today

datetime.datetime(2026, 2, 22, 16, 58, 37, 838453)

In [51]:
# Subtract the date column from today -> produces a timedelta Series
# Output format: 'X days HH:MM:SS.microseconds'
today - date['date']

0     2266 days 16:58:37.838453
1     2748 days 16:58:37.838453
2     2679 days 16:58:37.838453
3     2381 days 16:58:37.838453
4     2604 days 16:58:37.838453
                 ...           
995   2694 days 16:58:37.838453
996   2635 days 16:58:37.838453
997   2483 days 16:58:37.838453
998   2548 days 16:58:37.838453
999   2322 days 16:58:37.838453
Name: date, Length: 1000, dtype: timedelta64[ns]

In [52]:
# Extract only the whole-day count from the timedelta
# .dt.days returns an integer, discarding the time portion (hours, minutes, seconds)
(today - date['date']).dt.days

0      2266
1      2748
2      2679
3      2381
4      2604
       ... 
995    2694
996    2635
997    2483
998    2548
999    2322
Name: date, Length: 1000, dtype: int64

#### Month Passed 

In [54]:
np.round((today-date['date'])/np.timedelta64(1,'m'),0)

0      3264059.0
1      3958139.0
2      3858779.0
3      3429659.0
4      3750779.0
         ...    
995    3880379.0
996    3795419.0
997    3576539.0
998    3670139.0
999    3344699.0
Name: date, Length: 1000, dtype: float64

#### Elapsed Time in Minutes

  > Note: The heading says Month Passed but the code below computes **minutes** elapsed. Using np.timedelta64(1, m) divides the timedelta by one minute to yield total minutes as a float.

In [56]:
# Convert elapsed timedelta to total MINUTES (rounded to 0 decimal places)
# np.timedelta64(1, m) = one minute; dividing a timedelta by this yields minutes as float
np.round((today - date['date']) / np.timedelta64(1, 'm'), 0)

0      3264059.0
1      3958139.0
2      3858779.0
3      3429659.0
4      3750779.0
         ...    
995    3880379.0
996    3795419.0
997    3576539.0
998    3670139.0
999    3344699.0
Name: date, Length: 1000, dtype: float64

In [57]:
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   date    1000 non-null   object
 1   msg     1000 non-null   object
dtypes: object(2)
memory usage: 15.8+ KB


#### Converting to datetime datatype

In [59]:
# Convert the messages 'date' column from object -> datetime64[ns]
# This column has BOTH date AND time (e.g., '2013-12-15 00:50:00')
# pd.to_datetime() handles this combined format automatically
time['date'] = pd.to_datetime(time['date'])
time.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype         
---  ------  --------------  -----         
 0   date    1000 non-null   datetime64[ns]
 1   msg     1000 non-null   object        
dtypes: datetime64[ns](1), object(1)
memory usage: 15.8+ KB


#### Getting Hour , Minutes , Seconds

In [61]:
# Extract individual time components from the datetime column
# .dt.hour   -> integer hour (0-23, 24-hour clock)
# .dt.minute -> integer minute (0-59)
# .dt.second -> integer second (0-59)
time['hour'] = time['date'].dt.hour
time['minute'] = time['date'].dt.minute
time['sec'] = time['date'].dt.second
time.head()

,date,msg,hour,minute,sec
0,2013-12-15 00:50:00,ищу на сегодня мужика 37,0,50,0
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826,23,40,0
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576,0,21,0
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...,0,31,0
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше,23,11,0


#### Extracting Time Part

In [63]:
# Extract just the time portion (HH:MM:SS) as a Python time object
# Captures the full time in one column rather than separate hour/minute/second columns
# Note: resulting dtype is object (Python datetime.time objects), not datetime64
time['time'] = time['date'].dt.time
time.head()

,date,msg,hour,minute,sec,time
0,2013-12-15 00:50:00,ищу на сегодня мужика 37,0,50,0,00:50:00
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826,23,40,0,23:40:00
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576,0,21,0,00:21:00
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...,0,31,0,00:31:00
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше,23,11,0,23:11:00


---
## Time Difference for messages Dataset

Here we convert the elapsed timedelta to seconds, minutes, and hours using np.timedelta64.

| Desired Unit | Division |
|---|---|
| Seconds | np.timedelta64(1, s) |
| Minutes | np.timedelta64(1, m) |
| Hours   | np.timedelta64(1, h) |
| Days    | .dt.days |

### Raw Timedelta

In [65]:
# Compute raw elapsed time since each message was sent
# Output format: 'X days HH:MM:SS.microseconds'
today - time['date']

0     4452 days 16:08:37.838453
1     4316 days 17:18:37.838453
2     4802 days 16:37:37.838453
3     4104 days 16:27:37.838453
4     4501 days 17:47:37.838453
                 ...           
995   5091 days 16:08:37.838453
996   4412 days 17:44:37.838453
997   4877 days 17:21:37.838453
998   4993 days 17:24:37.838453
999   4265 days 17:33:37.838453
Name: date, Length: 1000, dtype: timedelta64[ns]

#### Time Diff in Second 

In [67]:
# Convert elapsed time to total SECONDS as a float
# Dividing a timedelta by np.timedelta64(1, s) yields total seconds
(today - time['date']) / np.timedelta64(1, 's')

0      3.847109e+08
1      3.729647e+08
2      4.149527e+08
3      3.546449e+08
4      3.889505e+08
           ...     
995    4.399205e+08
996    3.812607e+08
997    4.214353e+08
998    4.314579e+08
999    3.685592e+08
Name: date, Length: 1000, dtype: float64

#### In Minute

In [69]:
# Convert elapsed time to total MINUTES as a float
(today - time['date']) / np.timedelta64(1, 'm')

0      6.411849e+06
1      6.216079e+06
2      6.915878e+06
3      5.910748e+06
4      6.482508e+06
           ...     
995    7.332009e+06
996    6.354345e+06
997    7.023922e+06
998    7.190965e+06
999    6.142654e+06
Name: date, Length: 1000, dtype: float64

#### In Hours

In [71]:
# Convert elapsed time to total HOURS as a float
(today - time['date']) / np.timedelta64(1, 'h')

0      106864.143844
1      103601.310511
2      115264.627177
3       98512.460511
4      108041.793844
           ...      
995    122200.143844
996    105905.743844
997    117065.360511
998    119849.410511
999    102377.560511
Name: date, Length: 1000, dtype: float64

In [72]:
time.head()

,date,msg,hour,minute,sec,time
0,2013-12-15 00:50:00,ищу на сегодня мужика 37,0,50,0,00:50:00
1,2014-04-29 23:40:00,ПАРЕНЬ БИ ИЩЕТ ДРУГА СЕЙЧАС!! СМС ММС 0955532826,23,40,0,23:40:00
2,2012-12-30 00:21:00,Днепр.м 43 позн.с д/ж *.о 067.16.34.576,0,21,0,00:21:00
3,2014-11-28 00:31:00,КИЕВ ИЩУ Д/Ж ДО 45 МНЕ СЕЙЧАС СКУЧНО 093 629 9...,0,31,0,00:31:00
4,2013-10-26 23:11:00,Зая я тебя никогда не обижу люблю тебя!) Даше,23,11,0,23:11:00
